In [126]:
import pandas as pd

In [127]:
#Set the path of the two datasests. PATH1 = current year, PATH2 = historic
PATH1 = "../data/NYPD_Complaint_Data_Current__Year_To_Date__20231013.csv"
PATH2 = "../data/NYPD_Complaint_Data_Historic_20231013.csv"
RESULT = "../data/NYPD_Complaint_Data_Analysis.csv"


#Because Historic data is 3GB in size, to prevent multiple read operations we will read historic data once
#and store it for future processing.

##Get historic data with relevant columns
#usecols parameter is used because of the huge file size to reduce computation and memory bandwidth
historic_data = pd.read_csv(PATH2,usecols = ['ADDR_PCT_CD','BORO_NM','RPT_DT','OFNS_DESC','LAW_CAT_CD','PD_DESC','PREM_TYP_DESC']) 

#remove null data
historic_data = historic_data.dropna()

#we will keep historic complaints as a global data.
len(historic_data)

8352636

In [128]:
#In this exercise we are working with Bronx boro data. Get all pct under Bronx
bronx_pct = [40,41,42,43,44,45,46,47,48,49,50,52]

#Date range- Here we are analysing Week 23 data. 05-15-2023 to 06-11-2023
current_from_date='2023-05-15'
current_to_date ='2023-06-11'

historic_2022_from_date = '2022-05-15'
historic_2022_to_date = '2022-06-11'

seven_majors=["MURDER & NON-NEGL. MANSLAUGHTER", "RAPE", "ROBBERY", "FELONY ASSAULT", "BURGLARY","GRAND LARCENY","GRAND LARCENY OF MOTOR VEHICLE", "PETIT LARCENY"]

In [129]:
'''
    the function "extract_w23_data()" will extract the Week 23 data for a given PCT under a Given Boro
    parameters:
        1. path to current year data
        2. PCT (a PCT number)
        3. Boro (BRONX, QUEENS, MANHATTAN, BROOKLYN, STATEN ISLAND)
        
    
    return:
        returns a pandas dataframe that contains the Week-23 crime statistics for 2023 and 2022 for the given pct

    
    Note: extract_w23_data() calls "extract_historic_data()" to get historic data for the boro and pct.
'''

def extract_historic_data(boro,pct):
    
    
    #get data for the boro and pct
    historic_complaints = historic_data.loc[historic_data['BORO_NM'] == boro]
    historic_complaints = historic_complaints.loc[historic_complaints['ADDR_PCT_CD']==pct]
    

    #convert RPT_DT column to datetime column
    historic_complaints['RPT_DT']=pd.to_datetime(historic_complaints['RPT_DT'])
    
    #get complaints for the year 2022 between 2022-05-15 and 2022-06-11
    historic_w23_complaints= historic_complaints[(historic_complaints['RPT_DT'] >= historic_2022_from_date) & (historic_complaints['RPT_DT'] <= historic_2022_to_date)]
    
    
    
    #get total number of crimes under the seven major index crime. for this we will groupby the OFNS_DESC
    #first we will keep the the rows with major crimes in OFNS_DESC
    historic_w23_complaints_majors= historic_w23_complaints.loc[historic_w23_complaints['OFNS_DESC'].isin(seven_majors)]
    #Get total number of crimes per OFNS_DESC for the seven major crimes.
    historic_w23_major_ofns_count = historic_w23_complaints_majors.groupby('OFNS_DESC', as_index= False).size().rename(columns={'size':'2022'})
    
    
    
    #get total number of crimes for the non-major crimes. For this we will groupby the PD_DESC
    #first we will remove the the rows with major crimes in OFNS_DESC
    historic_w23_complaints_nonmajors = historic_w23_complaints.loc[~historic_w23_complaints['OFNS_DESC'].isin(seven_majors)]
    #Get total number of crimes per PD_DESC without the 7 major crimes.
    historic_w23_nonmajor_pd_desc_count = historic_w23_complaints_nonmajors.groupby('PD_DESC', as_index= False).size().rename(columns={'size':'2022'})
    
    
    #Now we will append the mjaor crime and non-major crime dataframes into a single dataframe
    historic_w23_major_ofns_count = historic_w23_major_ofns_count.rename(columns={"OFNS_DESC": "CRIME_DESC"})
    historic_w23_nonmajor_pd_desc_count = historic_w23_nonmajor_pd_desc_count.rename(columns={"PD_DESC": "CRIME_DESC"})
    
    
    historic_w23_crime_count = pd.concat([historic_w23_major_ofns_count, historic_w23_nonmajor_pd_desc_count]).sort_values(by=['2022'],ascending=False)
    
    return historic_w23_crime_count
    
    
    

def extract_w23_data(path1,pct,boro="BRONX"):
    
    ##Get data from current year
    current_complaints = pd.read_csv(path1)
    
    
    """ 
    For our analysis we are interested in a few columns, 
    ['ADDR_PCT_CD','BORO_NM','RPT_DT','OFNS_DESC','LAW_CAT_CD','PD_DESC','PREM_TYP_DESC']

    """
    
    #get relevant columns
    current_complaints=current_complaints[['ADDR_PCT_CD','BORO_NM','RPT_DT','OFNS_DESC','LAW_CAT_CD','PD_DESC', 'PREM_TYP_DESC']]
    
    #remove null data
    current_complaints = current_complaints.dropna()
    
    
    #get data for the boro and pct
    current_complaints = current_complaints.loc[current_complaints['BORO_NM'] == boro]
    current_complaints = current_complaints.loc[current_complaints['ADDR_PCT_CD']==pct]
    
    #if current_complaints returns 0 rows, then boro and pct combination is wrong.
    if(len(current_complaints)==0):
        return ("INVALID BORO AND PCT COMBINATION")

    
    #convert RPT_DT column to datetime column. I am choosing RPT_DT for the W23 date range, because it
    #represents the date when the complaint was reported. A crime reported today could happen anytime between
    #today and past months/years. So RPT_DT is the valid choice of date.
    current_complaints['RPT_DT']=pd.to_datetime(current_complaints['RPT_DT'])
    
    
    #get complaints for week 23
    current_w23_complaints= current_complaints[(current_complaints['RPT_DT'] >= current_from_date) & (current_complaints['RPT_DT'] <= current_to_date)]
    
    
    #get the crime category for w23 complaints for Major crimes
    current_w23_complaints_major_cat = current_w23_complaints.loc[current_w23_complaints['OFNS_DESC'].isin(seven_majors)]
    current_w23_complaints_major_cat = current_w23_complaints_major_cat[['OFNS_DESC','LAW_CAT_CD']]
    current_w23_complaints_major_cat = current_w23_complaints_major_cat.drop_duplicates(subset=['OFNS_DESC'])
    current_w23_complaints_major_cat = current_w23_complaints_major_cat.rename(columns={"OFNS_DESC": "CRIME_DESC"})
    
    #get the crime category for w23 complaints for Non-Major crimes 
    current_w23_complaints_nonmajor_cat= current_w23_complaints.loc[~current_w23_complaints['OFNS_DESC'].isin(seven_majors)]
    current_w23_complaints_nonmajor_cat = current_w23_complaints_nonmajor_cat[['PD_DESC','LAW_CAT_CD',]]
    current_w23_complaints_nonmajor_cat = current_w23_complaints_nonmajor_cat.drop_duplicates(subset=['PD_DESC'])
    current_w23_complaints_nonmajor_cat = current_w23_complaints_nonmajor_cat.rename(columns={"PD_DESC": "CRIME_DESC"})
    
    #get crime category for major crimes and non major crimes.
    current_w23_crime_cat = pd.concat([current_w23_complaints_major_cat, current_w23_complaints_nonmajor_cat])
        
    
    
    #get total number of crimes under the seven major index crime. for this we will groupby the OFNS_DESC
    #first we will keep the the rows with major crimes in OFNS_DESC
    current_w23_complaints_majors= current_w23_complaints.loc[current_w23_complaints['OFNS_DESC'].isin(seven_majors)]
    #Get total number of crimes per OFNS_DESC for the seven major crimes.
    current_w23_major_ofns_count = current_w23_complaints_majors.groupby('OFNS_DESC', as_index= False).size().rename(columns={'size':'2023'}).sort_values(by=['2023'],ascending=False)
    
    
    
    #get total number of crimes for the non-major crimes. For this we will groupby the PD_DESC
    #first we will remove the the rows with major crimes in OFNS_DESC
    current_w23_complaints_nonmajors = current_w23_complaints.loc[~current_w23_complaints['OFNS_DESC'].isin(seven_majors)]
    #Get total number of crimes per PD_DESC without the 7 major crimes.
    current_w23_nonmajor_pd_desc_count = current_w23_complaints_nonmajors.groupby('PD_DESC', as_index= False).size().rename(columns={'size':'2023'}).sort_values(by=['2023'],ascending=False)
    
    
    #Now we will append the mjaor crime and non-major crime dataframes into a single dataframe
    current_w23_major_ofns_count = current_w23_major_ofns_count.rename(columns={"OFNS_DESC": "CRIME_DESC"})
    current_w23_nonmajor_pd_desc_count = current_w23_nonmajor_pd_desc_count.rename(columns={"PD_DESC": "CRIME_DESC"})
    
    
    current_w23_crime_count = pd.concat([current_w23_major_ofns_count, current_w23_nonmajor_pd_desc_count]).sort_values(by=['2023'],ascending=False)
    
    

    
    ##Get data from previous year
    historic_w23_crime_count = extract_historic_data(boro, pct)
    
    #print(len(historic_w23_ofns_count))
    
    #Now I join the two crime totals, current year and previous year.
    ofns_stat = pd.merge(current_w23_crime_count,historic_w23_crime_count, on='CRIME_DESC')
    ofns_stat = pd.merge(ofns_stat,current_w23_crime_cat, on='CRIME_DESC')
    #print(current_w23_crime_cat)
    
    # getting Difference. Get the difference between 2023 and 2022 crimes. CurrentYear - PrevYear
    ofns_stat['Difference'] = ofns_stat['2023'] - ofns_stat['2022'] 
    
    #get % change between 2023 and 2022. (CurrentYear - PrevYear)/PrevYear * 100%
    ofns_stat['%Change'] =((ofns_stat['Difference']/ofns_stat['2022'])*100).round(1)
    
    #get final offense stat. This result should be similar to CompStat Result.
    ofns_stat = ofns_stat[['CRIME_DESC','LAW_CAT_CD','2023','2022','Difference','%Change']]
    
    return ofns_stat


#the output below is for PCT 44    
#pct_top_crime = extract_w23_data(PATH1,44)
#pct_top_crime.head(20)


,CRIME_DESC,LAW_CAT_CD,2023,2022,Difference,%Change
0,PETIT LARCENY,MISDEMEANOR,183,163,20,12.3
1,ASSAULT 3,MISDEMEANOR,121,113,8,7.1
2,"HARASSMENT,SUBD 3,4,5",VIOLATION,108,131,-23,-17.6
3,FELONY ASSAULT,FELONY,83,100,-17,-17.0
4,GRAND LARCENY,FELONY,59,80,-21,-26.2
5,"TRAFFIC,UNCLASSIFIED MISDEMEAN",MISDEMEANOR,56,1,55,5500.0
6,ROBBERY,FELONY,45,52,-7,-13.5
7,BURGLARY,FELONY,41,27,14,51.9
8,"HARASSMENT,SUBD 1,CIVILIAN",VIOLATION,41,42,-1,-2.4
9,GRAND LARCENY OF MOTOR VEHICLE,FELONY,39,36,3,8.3


In [130]:
#To find the top 5 crimes in each PCT, I followed the following algorithm.


#Rank crimes based on their ranking in Compstat. So 7 major crimes will get rank number according to their position.


"""

My Solution:

To give the top 5 crime problems in each pct, I have mainly focused on the number of occurrence for each crime in each pct.
Then I rank the crimes based on their types. 

    Assumption- Major crimes will have lower occurrence than non-major crimes.
    7 major crimes are rated according to their rank:
        Murder: 6,
        Rape: 5,
        Robbery: 3,
        Felony Assault: 2,
        Burglary: 2,
        GL: 2,
        GLA: 2
        
        LARCENY,PETIT FROM STORE-SHOPL and ASSAULT 3  and all other felonies have a rank number 1.
        All other misdemeanors have a rank number 0.7
        And violations 0.5

"""


crimes={
    "MURDER & NON-NEGL. MANSLAUGHTER":6,
    "RAPE": 5,
    "ROBBERY": 4,
    "FELONY ASSAULT":2, 
    "BURGLARY": 2,
    "GRAND LARCENY": 2,
    "GRAND LARCENY OF MOTOR VEHICLE": 2,
    "ASSAULT 3":1,
    "PETIT LARCENY": 0.8
}

cat={
   "MISDEMEANOR" : 0.7,
    "FELONY": 1,
    "VIOLATION": 0.5
}


In [131]:
def top_five_crimes(current_year_data_path,i):
    pct_top_crime = extract_w23_data(current_year_data_path,i)
   
    pct_top_crime['rank_order'] = pct_top_crime['CRIME_DESC'].apply(lambda x: crimes[x] if (x in crimes) else None)
    
    pct_top_crime['pd_order'] = pct_top_crime['LAW_CAT_CD'].apply(lambda x: cat[x])
    
   
    pct_top_crime['rank_order'] = pct_top_crime['rank_order'].fillna(pct_top_crime.pop('pd_order'))
    

    #calculate ranking for each crime type. ranking = number of occurance in 2023 * rank_order of the crime
    pct_top_crime['ranking']= pct_top_crime['2023'] * pct_top_crime['rank_order']
    
    #sort ranking in descending order and print the top 5 crimes
    return pct_top_crime.sort_values(by=['ranking'],ascending=False)
    
    
    

    
with open(RESULT,'a') as f:
    for i in bronx_pct:
        f.write(str(i)+"PCT\n")
        i_PCT_Top_Five = top_five_crimes(PATH1,i)
        
        #print output for 44 PCT as an example
        if(i==44): print(i_PCT_Top_Five.head(20))
        i_PCT_Top_Five = i_PCT_Top_Five[["CRIME_DESC","LAW_CAT_CD","2023","2022","Difference","%Change"]]
        i_PCT_Top_Five.head(20).to_csv(f)
        f.write("\n"*2)
    

                          CRIME_DESC   LAW_CAT_CD  2023  2022  Difference  \
6                            ROBBERY       FELONY    45    52          -7   
3                     FELONY ASSAULT       FELONY    83   100         -17   
0                      PETIT LARCENY  MISDEMEANOR   183   163          20   
1                          ASSAULT 3  MISDEMEANOR   121   113           8   
4                      GRAND LARCENY       FELONY    59    80         -21   
7                           BURGLARY       FELONY    41    27          14   
9     GRAND LARCENY OF MOTOR VEHICLE       FELONY    39    36           3   
2              HARASSMENT,SUBD 3,4,5    VIOLATION   108   131         -23   
5     TRAFFIC,UNCLASSIFIED MISDEMEAN  MISDEMEANOR    56     1          55   
10             MENACING,UNCLASSIFIED  MISDEMEANOR    39    30           9   
29                              RAPE       FELONY     5     1           4   
11           AGGRAVATED HARASSMENT 2  MISDEMEANOR    31    43         -12   